### 使用网格世界实现DQN算法

In [1]:
import numpy as np
import torch
from collections import deque
from torch import nn
import random
from rl_utils.GNGridWorldEnv import GridWorldEnv

# 定义神经网络
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128, output_dim=4):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

# 设备选择
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
    else "cpu"
)
print(f"使用设备: {device}")

# 设置环境
grid_world_size = 10
obstacle_count = 20
env = GridWorldEnv(size=grid_world_size, obstacle_count=obstacle_count)
n_actions = env.action_space.n

# 初始化参数
replay_buffer = deque(maxlen=2000)  # 经验回放缓冲区
batch_size = 64     # 批量大小
update_frequency = 100  # 目标网络更新频率
epsilon = 1.0       # 初始探索率
epsilon_min = 0.01  # 最小探索率
epsilon_decay = 0.995   # 探索率衰减
gamma = 0.99        # 折扣因子
lr = 0.001          # 学习率

# 网络初始化 (输入维度=2，对应x,y坐标)
main_network = NeuralNetwork(input_dim=2,hidden_dim=1024, output_dim=4).to(device)
target_network = NeuralNetwork(input_dim=2, hidden_dim=1024,output_dim=4).to(device)
target_network.load_state_dict(main_network.state_dict())
target_network.eval()  # 目标网络不需要梯度

# 损失函数和优化器
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(main_network.parameters(), lr=lr)

# ε-Greedy动作选择
def select_action(state, epsilon):
    if random.random() < epsilon:
        return random.randint(0, n_actions-1)

    state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    with torch.no_grad():
        q_values = main_network(state_tensor)
    return torch.argmax(q_values).item()

# 训练函数
def train():
    if len(replay_buffer) < batch_size:
        return None

    # 随机采样批次
    batch = random.sample(replay_buffer, batch_size)
    states, actions, rewards, next_states, dones = zip(*batch)

    # 转换为张量
    states = torch.tensor(np.array(states), dtype=torch.float32, device=device)
    actions = torch.tensor(actions, dtype=torch.long, device=device)
    rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
    next_states = torch.tensor(np.array(next_states), dtype=torch.float32, device=device)
    dones = torch.tensor(dones, dtype=torch.float32, device=device)

    # 计算当前Q值 (仅执行的动作)
    current_q_values = main_network(states)
    current_q_value = current_q_values.gather(1, actions.unsqueeze(1))

    # 计算目标Q值 (使用目标网络)
    with torch.no_grad():
        next_q_values = target_network(next_states)
        max_next_q = next_q_values.max(1)[0]
        target_q_values = rewards + (1 - dones) * gamma * max_next_q

    # 计算损失
    loss = loss_fn(current_q_value.squeeze(), target_q_values)

    # 优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

# 训练循环
global_step = 0
episode_rewards = []

for episode in range(1000):
    # 重置环境
    observation, _ = env.reset(seed=99,options={'enable_random_pos': True})
    state = observation['agent'] / grid_world_size  # 归一化到[0,1]
    done = False
    total_reward = 0
    step_count = 0

    while not done and step_count < 100:  # 最大步数限制
        # 选择动作
        action = select_action(state, epsilon)

        # 执行动作
        next_observation, reward, terminated, truncated, _ = env.step(action)
        next_state = next_observation['agent'] / grid_world_size
        done = terminated or truncated

        # 存储经验
        replay_buffer.append((state.copy(), action, reward, next_state.copy(), float(done)))

        # 训练
        train()

        # 更新状态
        state = next_state
        total_reward += reward
        step_count += 1
        global_step += 1

        # 定期更新目标网络
        if global_step % update_frequency == 0:
            target_network.load_state_dict(main_network.state_dict())

    # 更新探索率
    epsilon = max(epsilon * epsilon_decay, epsilon_min)
    episode_rewards.append(total_reward)

    # 打印进度
    if episode % 10 == 0:
        avg_reward = np.mean(episode_rewards[-10:])
        print(f"回合 {episode}, 平均奖励: {avg_reward:.2f}, Epsilon: {epsilon:.4f}, 步数: {global_step}")

env.close()

# 保存模型
torch.save(main_network.state_dict(), '../data/deep-q-learning/dqn-model.pth')

使用设备: cuda
回合 0, 平均奖励: -7.86, Epsilon: 0.9950, 步数: 100
回合 10, 平均奖励: -0.22, Epsilon: 0.9464, 步数: 822
回合 20, 平均奖励: -2.24, Epsilon: 0.9001, 步数: 1553
回合 30, 平均奖励: -4.30, Epsilon: 0.8561, 步数: 2265
回合 40, 平均奖励: -2.20, Epsilon: 0.8142, 步数: 2980
回合 50, 平均奖励: 0.03, Epsilon: 0.7744, 步数: 3715
回合 60, 平均奖励: 0.85, Epsilon: 0.7366, 步数: 4340
回合 70, 平均奖励: -0.97, Epsilon: 0.7005, 步数: 5152
回合 80, 平均奖励: 1.41, Epsilon: 0.6663, 步数: 5832
回合 90, 平均奖励: -3.32, Epsilon: 0.6337, 步数: 6645
回合 100, 平均奖励: -1.39, Epsilon: 0.6027, 步数: 7417
回合 110, 平均奖励: -8.45, Epsilon: 0.5733, 步数: 8191
回合 120, 平均奖励: 1.99, Epsilon: 0.5452, 步数: 8833
回合 130, 平均奖励: 4.43, Epsilon: 0.5186, 步数: 9308
回合 140, 平均奖励: 2.78, Epsilon: 0.4932, 步数: 9869
回合 150, 平均奖励: 1.90, Epsilon: 0.4691, 步数: 10444
回合 160, 平均奖励: 3.80, Epsilon: 0.4462, 步数: 10986
回合 170, 平均奖励: 6.67, Epsilon: 0.4244, 步数: 11295
回合 180, 平均奖励: 4.05, Epsilon: 0.4036, 步数: 11804
回合 190, 平均奖励: 8.22, Epsilon: 0.3839, 步数: 12025
回合 200, 平均奖励: 7.88, Epsilon: 0.3651, 步数: 12375
回合 210, 平均奖励: 7.79, E

### 使用模型进行预测

In [2]:
import time
import torch
import random
from torch import nn
from rl_utils.GNGridWorldEnv import GridWorldEnv

# 定义神经网络
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128, output_dim=4):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

# 设置环境
grid_world_size = 10
obstacle_count = 20
test_env = GridWorldEnv(size=grid_world_size, render_mode='human', obstacle_count=obstacle_count)
n_actions = test_env.action_space.n

epsilon_min = 0.01  # 最小探索率

# 测试学习到的策略
print("\n正在用学习到的策略运行测试 episode...")
observation, _ = test_env.reset(seed=99,options={'enable_random_pos': True})
state = observation['agent'] / grid_world_size
done = False
total_reward = 0
# 设备选择
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
    else "cpu"
)
print(f"使用设备: {device}")
# ε-Greedy动作选择
def select_action_test(state, epsilon):
    if random.random() < epsilon:
        return random.randint(0, n_actions-1)

    state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    with torch.no_grad():
        q_values = predict_network(state_tensor)
    return torch.argmax(q_values).item()

# 加载模型参数
model_path = '../data/deep-q-learning/dqn-model.pth'
print(f"正在从 {model_path} 加载模型...")
params = torch.load(model_path, map_location=device)

predict_network = NeuralNetwork(input_dim=2,hidden_dim=1024, output_dim=4).to(device)
predict_network.load_state_dict(params)
predict_network.eval()  # 设置为评估模式

while not done:
    action = select_action_test(state, epsilon_min)  # 使用最小探索率
    next_observation, reward, terminated, truncated, _ = test_env.step(action)
    state = next_observation['agent'] / grid_world_size
    total_reward += reward
    done = terminated or truncated
    time.sleep(0.3)

print(f"测试 episode 总奖励: {total_reward:.2f}")
test_env.close()



正在用学习到的策略运行测试 episode...
使用设备: cuda
正在从 ../data/deep-q-learning/dqn-model.pth 加载模型...


/home/jiangchengxuan/anaconda3/envs/d2l/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


测试 episode 总奖励: 9.92
